In [ ]:
!wget https://raw.githubusercontent.com/google/BIG-bench/main/bigbench/benchmark_tasks/odd_one_out/task.json

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import torch
import asyncio
import json
import sys
sys.path.append("../../")

from lmql.model.client import ServedPretrainedModel

In [18]:
with open("task.json") as f:
    data = json.load(f)

In [4]:
data.keys()

dict_keys(['canary', 'name', 'description', 'keywords', 'preferred_score', 'append_choices_to_input', 'metrics', 'examples'])

In [6]:
data["examples"][0]

{'input': 'Pick the odd word out: white, black, red, apple',
 'target_scores': {'white': 0, 'black': 0, 'red': 0, 'apple': 1}}

In [ ]:
model = ServedPretrainedModel("http://localhost:8080", "gpt2")

bos_token_id = (await model.tokenize("<BOS>"))[0]
eos_token_id = (await model.tokenize("<EOS>"))[0]

In [41]:
async def run_eval():
    for d in data["examples"]:
        for o in d["target_scores"].keys():
            full_prompt = d["input"] + ". The odd one is " + o
            input_ids = await model.tokenize(full_prompt)
            input_ids = torch.tensor(input_ids).long()
            print(input_ids.shape)
            res = await model.generate(input_ids=input_ids.unsqueeze(0), max_length=1024, 
                                       early_termination=True, num_return_sequences=1, 
                                       max_new_tokens=20, do_sample=True,
                                       early_stopping=True, no_repeat_ngram_size=3, 
                                       eos_token_id=eos_token_id, bos_token_id=bos_token_id)
            print(res)
        print(d)
        # input_ids = await model.tokenize("A: What is the circumference of the earth?\n B: I believe the best person to answer this question is the  Bible itself. Q: What does the Bible say about the size of the universe?.\n Indeed, in a recent interview the  Bible itself. Q: What does the Bible say about the size of the universe? addressed this question:")
        # input_ids = torch.tensor(input_ids).view(1, -1)
        # res = await model.generate(input_ids=input_ids, max_length=1024, early_termination=True, num_return_sequences=1, max_new_tokens=20, do_sample=True, num_beams=3, early_stopping=True, no_repeat_ngram_size=3, eos_token_id=eos_token_id, bos_token_id=bos_token_id)
        break

await run_eval()
        
# input_ids = await model.tokenize("A: What is the circumference of the earth?\n B: I believe the best person to answer this question is the  Bible itself. Q: What does the Bible say about the size of the universe?.\n Indeed, in a recent interview the  Bible itself. Q: What does the Bible say about the size of the universe? addressed this question:")
# input_ids = torch.tensor(input_ids).view(1, -1)
# res = await model.generate(input_ids=input_ids, max_length=1024, early_termination=True, num_return_sequences=1, max_new_tokens=20, do_sample=True, num_beams=3, early_stopping=True, no_repeat_ngram_size=3, eos_token_id=eos_token_id, bos_token_id=bos_token_id)

CancelledError: 